In [13]:
import torch
import torchvision
from torch.optim import Adam
from PIL import Image
from torchvision import transforms

In [14]:
img = Image.open("oxford-iiit-pet/images/Abyssinian_1.jpg").convert("RGB")

In [15]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),             # Converts to [0,1] tensor (C,H,W)
    transforms.Normalize(              # Standard ImageNet normalization
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [16]:
img_tr = transform(img)

In [17]:
img_tr.shape

torch.Size([3, 224, 224])

In [ ]:
from models.model_blocks import UNetEncoder
encodeer = UNetEncoder()